In [9]:
import os
import keyring
import glob
from ftplib import FTP
from arcgis import GIS
# from arcgis.features import GeoAccessor
# from arcgis.features import GeoSeriesAccessor
# from arcgis.features import FeatureLayerCollection
# from arcgis.features import FeatureLayer


# Toggles

In [2]:
# Switch to false to disable
update_maps = True
update_charts = True

# Update files in arcgis online

In [ ]:
# Sign into ArcGIS Online
un = 'analytics_wfrc'
pw = keyring.get_password('Analytics AGOL', un)
gis = GIS(username=un, password=pw)

In [ ]:
# store path to the new data
cityarea_zip = r".\Outputs\map\CITYAREA.gdb.zip"
county_zip = r".\Outputs\map\COUNTY.gdb.zip"
distmed_zip = r".\Outputs\map\DISTMED.gdb.zip"
otherarea_zip = r".\Outputs\map\OTHERAREA.gdb.zip"
taz_zip = r".\Outputs\map\TAZ.gdb.zip"

In [ ]:
# store item ids
cityarea_item_id = '265bfbe2cae7490196e230eced9109b2'
county_item_id = 'cb97017428fe470280fd015b4d7d61bf'
distmed_item_id = 'bdf485e1e5e94c16b08b2464bc4f50a5'
otherarea_item_id = '56ae4b1483e54eff932c24127d496f46'
taz_item_id = '844f78208e604aabb0d14d5ae827712d'

In [ ]:
def update_and_republish_item(id, new_zip):
    item = gis.content.get(id)

    item.update(data=new_zip) # store this result and test, while loop?

    item.publish(overwrite=True) # store this result and test, while loop?

In [ ]:
# get the item using the id
cityarea_item = gis.content.get(cityarea_item_id)
county_item = gis.content.get(county_item_id)
distmed_item = gis.content.get(distmed_item_id)
otherarea_item = gis.content.get(otherarea_item_id)
taz_item = gis.content.get(taz_item_id)

In [ ]:
if update_maps == True:
    
    # update the data source, then republish
    print('updating city area')
    cityarea_item.update(data=cityarea_zip)
    cityarea_item.publish(overwrite=True)

    print('updating county')
    county_item.update(data=county_zip)
    county_item.publish(overwrite=True)

    print('updating medium district')
    distmed_item.update(data=distmed_zip)
    distmed_item.publish(overwrite=True)

    print('updating other area')
    otherarea_item.update(data=otherarea_zip)
    otherarea_item.publish(overwrite=True)

    print('updating taz')
    taz_item.update(data=taz_zip)
    taz_item.publish(overwrite=True)

In [ ]:
# ## Access the feature layer as a GIS content item
# item_id = 'adf71d1f547b4a5581afc41328deeebd'
# item = gis.content.get(item_id)
# # feature_layer = item.layers[0]

# v1 = r"E:\Projects\REMM-Process-SE-Output-For-Web\Outputs\C_TEST_1.gdb.zip"
# v2 = r"E:\Projects\REMM-Process-SE-Output-For-Web\Outputs\New\C_TEST_1.gdb.zip"

# item.update(data=v2)
# item.publish(overwrite=True)

# Update files on server

In [11]:
local_chart_folder = r'.\Outputs\chart'
un2= 'wfrcorg'
pw2 = keyring.get_password('WFRC File Server', un2)
ip = keyring.get_password('WFRC File Server IP', un2)
ftp = FTP(ip)
ftp.login(user=un2, passwd =pw2)

'230 OK. Current restricted directory is /'

In [12]:
if update_charts == True:
    geogs = ['CITYAREA', 'COUNTY', 'DISTMED', 'OTHERAREA', 'TAZ']

    for geog in geogs:
        ftp.cwd(f'/public_html/household-job-forecast-map/widgets/SESidebar/data/chart/{geog}')
        
        old_jsons = [item for item in ftp.nlst() if item not in ['.','..'] and item.endswith('.json')]
        if len(old_jsons) > 0:
            print(f'deleting old jsons ({geog})...')
            for json in old_jsons:
                # print(json)
                ftp.delete(json)
        else:
            print(f'no jsons to delete ({geog})...')

        print(f'uploading new jsons ({geog})...')
        new_jsons = glob.glob(os.path.join(local_chart_folder, geog, '*.json'))
        
        for json in new_jsons:
            #print(json)

            with open(json,'rb') as file:
                ftp.storbinary('STOR ' + os.path.basename(json), file)

deleting old jsons CITYAREA...
uploading new jsons CITYAREA...
.\Outputs\chart\CITYAREA\CityArea_Alpine.json
.\Outputs\chart\CITYAREA\CityArea_Alta.json
.\Outputs\chart\CITYAREA\CityArea_American Fork.json
.\Outputs\chart\CITYAREA\CityArea_Balance of BOX ELDER.json
.\Outputs\chart\CITYAREA\CityArea_Benjamin.json
.\Outputs\chart\CITYAREA\CityArea_Bluffdale.json
.\Outputs\chart\CITYAREA\CityArea_Bountiful.json
.\Outputs\chart\CITYAREA\CityArea_Box Elder County North.json
.\Outputs\chart\CITYAREA\CityArea_Box Elder County South.json
.\Outputs\chart\CITYAREA\CityArea_Brigham City.json
.\Outputs\chart\CITYAREA\CityArea_Brighton.json
.\Outputs\chart\CITYAREA\CityArea_Camp Williams.json
.\Outputs\chart\CITYAREA\CityArea_Cedar Fort.json
.\Outputs\chart\CITYAREA\CityArea_Cedar Hills.json
.\Outputs\chart\CITYAREA\CityArea_Centerville.json
.\Outputs\chart\CITYAREA\CityArea_Clearfield.json
.\Outputs\chart\CITYAREA\CityArea_Clinton.json
.\Outputs\chart\CITYAREA\CityArea_Copperton.json
.\Outputs\cha

In [13]:
# close the connection
ftp.quit()

'221-Goodbye. You uploaded 52730 and downloaded 0 kbytes.\n221 Logout.'